**AI & Machine Learning (KAN-CINTO4003U) - Copenhagen Business School | Spring 2025**

***

# Part II: Agents

Please see the description of the assignment in the README file (section 1) <br>
**Guide notebooks**: [guides/router_agents_guide.ipynb](guides/agents_guide.ipynb) and [guides/tool_agents_guide.ipynb](guides/tool_agents_guide.ipynb)


***
<br>

* Play around with the agent systems in the guides. Maybe try to create a similar system with new nodes? Or you can try to add new tools to the existing tool-using agent system.

* Remember to include some reflections on your results. Are there, for example, any hyperparameters that are particularly important?

* You should follow the steps given in the `router_agents_guide` notebook or the `tool_agents_guide` notebook to create your own agent system - or try to optimize the existing ones.

<br>

***

### IT Support Ticket Agent

This agent processes incoming IT support emails and determines whether they are urgent. If urgent, it notifies the IT support manager. Otherwise, it requests the user to create a formal ticket.

In [99]:
# built-in libraries
from typing import Literal, TypedDict,  Any, Optional

# langgraph libraries
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables.graph import  MermaidDrawMethod

# misc libraries
from pydantic import Field
from IPython.display import Image
from decouple import config
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from tqdm import tqdm
from sklearn.metrics import f1_score

# local modules
from src.llm import LLMCaller

In [100]:
WX_API_KEY = config("WX_API_KEY")
WX_PROJECT_ID = config("WX_PROJECT_ID")
WX_API_URL = "https://us-south.ml.cloud.ibm.com"

In [101]:
model = LLMCaller(
    api_key=WX_API_KEY,
    project_id=WX_PROJECT_ID,
    api_url=WX_API_URL,
    model_id="watsonx/ibm/granite-3-2-8b-instruct",
    params={
        GenParams.TEMPERATURE: 0.7,
        GenParams.MAX_NEW_TOKENS: 300,
    }
)

In [102]:
response = model.invoke(prompt="Write a haiku about a fish")

response

BaseResponse(answer="Silent swimmer gleams,\nIn water's depths, it silently dreams,\nBeside the undulating streams.")

In [103]:
class ITSupportState(TypedDict):
    email: dict[str, Any]  # Incoming email details
    is_urgent: Optional[bool]  # Whether the issue is urgent
    summary: Optional[str]  # Condensed summary of the issue
    response: Optional[str]  # Response to the user

In [104]:
def analyze_ticket(state: ITSupportState):
    email = state['email']
    prompt = f"""Analyze the following email and determine if the issue is urgent.
    Provide a short summary if urgent, or suggest a polite response asking the user to create a formal ticket.
    Email:
    From: {email['sender']}
    Subject: {email['subject']}
    Body: {email['body']}"""

    response_model = model.create_response_model(
        "TicketAnalysis",
        {
            "is_urgent": (bool, Field(description="Whether the issue is urgent.")),
            "summary": (Optional[str], Field(description="Condensed summary of the issue.")),
            "response": (Optional[str], Field(description="Response to the user."))
        }
    )

    response = model.invoke(prompt, response_model=response_model)
    return {
        "is_urgent": response.is_urgent,
        "summary": response.summary,
        "response": response.response
    }

In [105]:
def notify_manager(state: ITSupportState):
    if state['is_urgent'] and state['summary']:
        print(f"Notifying IT support manager: {state['summary']}")
    return {}

In [106]:
def respond_to_user(state: ITSupportState):
    if not state['is_urgent'] and state['response']:
        print(f"Sending response to user: {state['response']}")
    return {}

In [107]:
def route_ticket(state: ITSupportState) -> str:
    return "notify_manager" if state['is_urgent'] else "respond_to_user"

### Visualizing the IT Support Ticket Agent



In [108]:
# Create the graph
ticket_graph = StateGraph(ITSupportState)  # Initialize the graph with the ITSupportState state

# Add nodes
ticket_graph.add_node("analyze_ticket", analyze_ticket)  # Add the analyze_ticket node
ticket_graph.add_node("notify_manager", notify_manager)  # Add the notify_manager node
ticket_graph.add_node("respond_to_user", respond_to_user)  # Add the respond_to_user node

# Start the edges - LangGraph requires a start node
ticket_graph.add_edge(START, "analyze_ticket")  # Start with analyzing the ticket

# Add conditional branching from analyze_ticket
ticket_graph.add_conditional_edges(
    "analyze_ticket",
    route_ticket,  # Based on this function..
    {
        "notify_manager": "notify_manager",  # .. we route to these nodes
        "respond_to_user": "respond_to_user"
    }
)

# Add the final edges
ticket_graph.add_edge("notify_manager", END)  # End the process after notifying the manager
ticket_graph.add_edge("respond_to_user", END)  # End the process after responding to the user

# Compile the graph
compiled_ticket_graph = ticket_graph.compile()  # Compile the graph

# Display the graph structure in console
print("IT Support Ticket Agent Graph:")

IT Support Ticket Agent Graph:


In [109]:
Image(compiled_ticket_graph.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.API,max_retries=5, retry_delay=2.0))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

## Testing the agent

In [ ]:
test_email = {
    "sender": "user@example.com",
    "subject": "System Down!",
    "body": "Our main server is down, and we need immediate assistance."
}

compiled_ticket_graph.invoke({
    "email": test_email
})

Notifying IT support manager: Server outage requiring immediate attention.


{'email': {'sender': 'user@example.com',
  'subject': 'System Down!',
  'body': 'Our main server is down, and we need immediate assistance.'},
 'is_urgent': True,
 'summary': 'Server outage requiring immediate attention.',
 'response': 'Apologies for the inconvenience. Please create a formal ticket by providing as many details as possible about the issue. This will help our team address the situation more effectively.'}

In [ ]:
test_email = {
    "sender": "user@example.com",
    "subject": "Mouse lagging",
    "body": "Hi, my mouse is kid of lagging and is a bit annoing. It happens a couple a times a day"
}

compiled_ticket_graph.invoke({
    "email": test_email
})

Sending response to user: Dear user, thank you for reaching out about your mouse lagging issue. While it's not an immediate emergency, it's important to resolve this for a smooth experience. Please consider creating a formal ticket detailing the problem, including the time of day it typically occurs and any recent changes to your system or software. This will help us better understand and address the issue. In the meantime, try restarting your computer to see if that resolves the issue.


{'email': {'sender': 'user@example.com',
  'subject': 'Mouse lagging',
  'body': 'Hi, my mouse is kid of lagging and is a bit annoing. It happens a couple a times a day'},
 'is_urgent': False,
 'summary': 'Mouse lagging issue',
 'response': "Dear user, thank you for reaching out about your mouse lagging issue. While it's not an immediate emergency, it's important to resolve this for a smooth experience. Please consider creating a formal ticket detailing the problem, including the time of day it typically occurs and any recent changes to your system or software. This will help us better understand and address the issue. In the meantime, try restarting your computer to see if that resolves the issue."}

## Reflections
The image/graph for the agent nodes does not work, but I cant seem to fix the issue. To test effictively, there would need to be more classified examples to evaluate from. This is next step of agent is developed further.